In [1]:
# Make sure accelerate is installed otherwise the following error may be raised
"""
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. 
Defaulting to `low_cpu_mem_usage=False`. 
It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. 
You can do so with: 
    ```
    pip install accelerate
    ```
"""

import os
import gc
import diffusers
import schedulers
import inspect
import numpy as np
import seaborn as sns
from dataclasses import dataclass
from tqdm.auto import tqdm
from math import ceil

import torch
from torchvision import transforms as tfms

from autoencoders import AutoEncoder, VAE

from diffusers import AutoencoderKL, LMSDiscreteScheduler, UNet2DConditionModel
from transformers import CLIPTextModel, CLIPTokenizer, logging
from huggingface_hub import notebook_login

from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image

# TODO this will have to be changed for the main.py code
if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

# Supress CLIPTextModel warnings
logging.set_verbosity_error()

c:\Users\diogo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class StableDiffusionPipeline():

    def __init__(self, prompt, inference_steps, guidance_scale,
                 height=512, width=512,
                 scheduler="LMSDiscreteScheduler", scheduler_params={},
                 seeded_generator=torch.manual_seed(32)):
        self.prompt = prompt

        self.inference_steps = inference_steps
        self.guidance_scale = guidance_scale

        self.height = height
        self.width = width

        self.seeded_generator = seeded_generator

        # Device
        self.initialise_device()

        # Components
        self.initialise_components(scheduler, scheduler_params)
        self.components_to_gpu()

        # Initialise embeddings, scheduler and latents that are required prior to inference
        self.initialise_text_embeddings(self.prompt)
        self.initialise_scheduler()
        self.initialise_latents()

    def initialise_device(self):
        # Set device based on CUDA, MPS (MacOS), CPU
        self.device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

        if "mps" == self.device: 
            os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

    def initialise_components(self, scheduler, scheduler_params):
        # In the paper our q() - the distribution over a noisey object at time step t has a closed form solution
        # 
        self.scheduler = self.select_scheduler(scheduler, scheduler_params)
        
        # Load the autoencoder model which will be used to decode the latents into image space.
        self.vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

        # # Loading a previously trained model using autoencoders.py
        # state_dict = torch.load("../training/saved_models/vae.pth")
        # # Load the state_dict into the model
        # self.vae = VAE()
        # self.vae.load_state_dict(state_dict)
        # # Make sure that we set the model to evaluation mode to make sure that the running statistics that were calculated in batch normalisation
        # # are consistent during inference
        # self.vae.eval()

        # The UNet model for generating the latents.
        self.unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")
        
        # Load the tokeniser and text encoder to tokenize and encode the text.
        # CLIP uses a ViT like transformer that gets visual features and a causal language model for the text features
        # Both the text and visual features are proejcted into a latent space with  identical dimensions and a dot product between the projected image and the text features is used as a similarity score
        # Images are split into fixed-size and non overlapping sequendes called patches - positional embeddings are also added.
        # If needed images can be resized and normalised using the CLIPImageProcessor
        # The CLIPTokeniser encodes the text
        self.tokeniser = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
        self.text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")


    def select_scheduler(self, scheduler_type: str, scheduler_params: dict=None):
        # Try to get the scheduler from the diffusers library
        try:
            SchedulerClass = getattr(diffusers, scheduler_type)
        except AttributeError:
            try:
                # If it is not in the diffusers library then we can check our schedulers module that uses the SchedulerMixin
                SchedulerClass = getattr(schedulers, scheduler_type)
            except AttributeError:
                print(self.get_custom_schedulers())
                raise AttributeError(f"The following scheduler {scheduler_type} does not exist in the diffusers library or in the scheduler file.\n The following are possible choices in the diffusers library:\n{self.get_diffusers_schedulers()}")


        self.validate_scheduler_params(SchedulerClass, scheduler_params)
        # Instantiate the scheduler with the provided parameters - make sure that your parameters that you pass in match the requirements
        return SchedulerClass(**scheduler_params)

    def validate_scheduler_params(self, scheduler_class, scheduler_params):
        # Get the __init__ method of the scheduler class
        init_signature = inspect.signature(scheduler_class.__init__)
        
        # Get the parameter names (excluding 'self')
        valid_params = [param for param in init_signature.parameters if param != 'self']

        # Identify invalid parameters
        invalid_params = [param for param in scheduler_params if param not in valid_params]

        if invalid_params:
            print(f"Error: Invalid parameters {invalid_params} for {scheduler_class.__name__}.")
            print(f"Expected parameters are: {valid_params}")
            return False
        return True

    def get_diffusers_schedulers(self):
        # We can use the inspect module if we get all the members of the diffusers module and we check if it ends with Scheduler
        return [name for name, obj in inspect.getmembers(diffusers) if inspect.isclass(obj) and name.endswith('DiscreteScheduler')]
    
    def get_custom_schedulers(self):
        # We can use the inspect module if we get all the members of the diffusers module and we check if it ends with Scheduler
        return [name for name, obj in inspect.getmembers(schedulers) if inspect.isclass(obj) and name.endswith('Scheduler')]

    def components_to_gpu(self):
        # Move the initialised components to the gpu
        self.vae = self.vae.to(self.device)
        self.text_encoder = self.text_encoder.to(self.device)
        self.unet = self.unet.to(self.device)

    def initialise_text_embeddings(self, prompt):
        # Conditioned text embeddings
        self.prompt_tokens = self.tokeniser(prompt, padding="max_length", max_length=self.tokeniser.model_max_length, truncation=True, return_tensors="pt")
        with torch.no_grad():
            # text_embeddings = self.text_encoder(text_input.input_ids.to(self.device))[0]
            text_embeddings = self.get_embeddings(self.prompt_tokens.input_ids.to(self.device))
        
        # Matching the conditioned max max length and creating the unconditioned embeddings
        max_length = self.prompt_tokens.input_ids.shape[-1]
        uncond_input = self.tokeniser([""], padding="max_length", max_length=max_length, return_tensors="pt")
        with torch.no_grad():
            uncond_embeddings = self.text_encoder(uncond_input.input_ids.to(self.device))[0]

        # Concatenate both
        self.text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

    def visualise_tokenisation(self, show_tensor=True, show_attention_mask=True):
        # Starts off with a prompt - then we tokenise it into a series of tokens using our CLIPTokeniser
        # This tokeniser makes sure it is padded
        printed_tokens = set()

        if show_tensor or show_attention_mask:
            if show_tensor:
                print(f"Tokenised text tensor:\n{self.prompt_tokens['input_ids'][0]}\n")
            if show_attention_mask:
                print(f"Tokenised text tensor:\n{self.prompt_tokens['attention_mask'][0]}\n")
        else:
            print("For the sake of clarity the padded tokens and the attention mask are removed from the output, run this method with show_tensor=True and/or attention_mask=True to see them respectively.\n")

        # See the individual tokens
        for token in self.prompt_tokens['input_ids'][0]:
            token_value = int(token)
            if token_value not in printed_tokens:
                print(f"Token: {token} - Value: {self.tokeniser.decoder.get(token_value)}")
                printed_tokens.add(token_value)

    def build_causal_attention_mask(self, bsz, seq_len, dtype):
        # lazily create causal attention mask, with full attention between the vision tokens
        # pytorch uses additive attention mask; fill with -inf
        mask = torch.empty(bsz, seq_len, seq_len, dtype=dtype)
        mask.fill_(torch.tensor(torch.finfo(dtype).min))
        mask.triu_(1)  # zero out the lower diagonal
        mask = mask.unsqueeze(1)  # expand mask
        return mask

    def get_embeddings(self, input_embeddings):
        # TODO undestand code and comment it
        token_embeddings = self.text_encoder.text_model.embeddings.token_embedding(input_embeddings)
        # TODO why only 77
        position_ids = self.text_encoder.text_model.embeddings.position_ids[:, :77]
        positional_embeddings = self.text_encoder.text_model.embeddings.position_embedding(position_ids)

        embedding = token_embeddings + positional_embeddings

        # CLIP's text model uses causal mask, so we prepare it here:
        bsz, seq_len = embedding.shape[:2]
        causal_attention_mask = self.build_causal_attention_mask(bsz, seq_len, embedding.dtype)

        # Getting the output embeddings involves calling the model with passing output_hidden_states=True
        # so that it doesn't just return the pooled final predictions:
        encoder_outputs = self.text_encoder.text_model.encoder(
            inputs_embeds=embedding,
            attention_mask=None,
            causal_attention_mask=causal_attention_mask.to(self.device),
            output_attentions=None,
            output_hidden_states=True,
            return_dict=None,
        )

        # We're interested in the output hidden state only
        output = encoder_outputs[0]

        # There is a final layer norm we need to pass these through
        output = self.text_encoder.text_model.final_layer_norm(output)
        return output

    def initialise_scheduler(self):
        # The timesteps - betas that we define are values that are between 0 and 1 and change how the distribution is changed
        # When we initially start with a beta value of 0 it starts off with the image that we started with and 0 noise applied
        # It makes sense to scale the variance (the amount of noise that we add) however we must make sure that we also vary the 
        # mean of the image being added - we have to make sure that as the variance grows we scale the mean down by 
        # roo(1 - beta) so that our distribution remains and our pure white noise at the end of our sampling is a standard normal distribution
        self.scheduler.set_timesteps(self.inference_steps)
        self.scheduler.timesteps = self.scheduler.timesteps.to(torch.float32)

    def visualise_scheduler(self, scheduler=None, timesteps=None):
        # TODO figure out how scheduler.scale_model_input works and what it does (what scaling / pre conditioning is)
        if not scheduler:
            scheduler = self.scheduler

        if timesteps:
            scheduler.set_timesteps(timesteps)

        # Visualising the sigmas (the amount of noise) that the scheduler will choose to add to our latents
        plt.plot(scheduler.sigmas)
        plt.title('Noise Schedule')
        plt.xlabel('Steps')
        plt.ylabel('Sigma')
        plt.show()

    def visualise_timestep_embeddings(self):
        _, ax = plt.subplots(1, 1, figsize=(8,6))
        sns.heatmap(self.scheduler.timesteps, cmap='bwr', ax=ax)
        ax.set_xlabel('Embedding Dimension')
        ax.set_ylabel('Timestep')
        plt.show()

    def visualise_noised_latents(self, latents, scheduler, timesteps):
        max_timesteps = len(scheduler.timesteps)
        assert timesteps < max_timesteps, "The timestep chosen to visualise the noised image must be lower than the current max {max_timesteps}"

        noise = torch.rand_like(latents)
        # This add noise function is the same noised_sample = original + noise * sigmas
        encoded_and_noised = scheduler.add_noise(latents, noise, timesteps=torch.tensor([scheduler.timesteps[timesteps]]))
        decoded_latents = self.decode_latents(encoded_and_noised.float().to("cuda"))
        plt.imshow(self.scale_image(decoded_latents))

    def initialise_latents(self):
        # Prep latents - our latents are 64x smaller so we generate the noise in the latent dimension
        # We can generate an object by sampling white noise from a standard normal distribution adn then iteratively sampling x_t-1
        self.latents = torch.randn(
            (1, self.unet.config.in_channels, self.height // 8, self.width // 8),
            generator=self.seeded_generator,
        )
        self.latents = self.latents.to(self.device)
        self.latents = self.latents * self.scheduler.init_noise_sigma # Scaling (previous versions did latents = latents * self.scheduler.sigmas[0]

    def update_prompt(self, prompt):
        self.prompt = prompt
        self.initialise_text_embeddings(self.prompt)

    def image_to_latents(self, image):
        # Output the latents of a single image
        with torch.no_grad():
            # TODO break down this scaling into separate components and explain it
            latent = self.vae.encode(tfms.ToTensor()(image).unsqueeze(0).to(self.device)*2-1) # Note scaling
        images = 0.18215 * latent.latent_dist.sample()
        images = images.to("cpu")
        self.display_images(np.array(images[0]), 4, cmap="gray")
        return images

    def decode_latents(self, latents):
        # TODO figure out why these are scaled scale and decode the image latents with vae
        print(latents.shape)
        latents = 1 / 0.18215 * latents
        print(latents.shape)
        with torch.no_grad():
            return self.vae.decode(latents[0]).sample

    def scale_image(self, image):
        # TODO understand and comment why this is done
        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
        image = (image * 255).round().astype("uint8")
        return Image.fromarray(image[0])

    def show_generated_image(self):
        plt.imshow(self.image)

    def display_images(self, images, cols=1, **kwargs):
        num_images = len(images)
        rows = ceil(num_images / cols)

        # Create subplots and flatten it so that we get an easy iterable
        _, axs = plt.subplots(rows, cols, figsize=(15, 5*rows))
        axs = axs.flatten()
        
        # Iterate through the images and plot them - making sure to turn off the axis
        for i, image in enumerate(images):
            axs[i].imshow(image, **kwargs)
            axs[i].axis('off')

        # We can use the remainder of the i variable to hide any unused subplots
        for j in range(i + 1, len(axs)):
            axs[j].axis('off')

        # Adjust layout to prevent overlapping
        plt.tight_layout()
        plt.show()

    def generate(self, show=True):
        # with autocast("cuda"):
        for _, t in tqdm(enumerate(self.scheduler.timesteps), total=len(self.scheduler.timesteps)):
            # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
            latent_model_input = torch.cat([self.latents] * 2)

            # By preconditioning or scaling the latents we are ensuring that we are improving the stability and performance of the model by compensating for the different noise levels
            # and adjusting the variability of the magnitude of the inputs as the sigma will always be chosen by the scheduler
            latent_model_input = self.scheduler.scale_model_input(latent_model_input, t)

            # predict the noise residual
            with torch.no_grad():
                # The encoder hidden states in the unet is the encoded text embeddings that CLIP provides that will guide the model
                noise_pred = self.unet(latent_model_input, t, encoder_hidden_states=self.text_embeddings).sample

            # Guidance - we initially combined the two text embeddings and concatenated two latents to do the conditioned and unconditioned together
            # Use .chunk to split the noise prediction of the two concatenated tensors into the unconditioned noise prediction
            # and the conditioned on text noise prediction, these are then updated using the guidance scale formula
            # Take the difference between the conditioned and the unconditioned noise - the higher the guidance scale the more we follow the conditioned noise
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + self.guidance_scale * (noise_pred_text - noise_pred_uncond)

            # Here we are essentially sampling from the posterior distribution in order to remove the noise
            self.latents = self.scheduler.step(noise_pred, t, self.latents).prev_sample

        self.image = self.decode_latents(self.latents)
        self.image = self.scale_image(self.image)
        if show == True:
            self.show_generated_image()

        # self.clear_cuda()

        return self.image

    def clear_cuda(self):
        # Clearing CUDA so that it does not run out of memory during multiple instantiations - big problem especially in notebooks
        torch.cuda.synchronize()
        gc.collect()
        torch.cuda.empty_cache()
        

In [7]:
# TODO create a config like GPT
@dataclass
class SDPipelineConfig:
    """
    Stable Diffusion Pipeline Config

    prompt: The text prompt you want to condition the image generation on
    height: Height of image generated                            
    width: Width of image generated                              
    inference_steps: Number of diffusion steps that you want to take place                   
    guidance_scale: Guidance value that affects how closely the prompt affects diffusion                        
    seed: Random generation seed to keep results reproducible
    """
    prompt: str = "A parrot"
    height: int = 512                              
    width: int = 512                                 
    inference_steps: int = 15                         
    guidance_scale: float = 7.5                        
    seed: int = 42

    # TODO see if any assertions can be added here to make sure the values are good e.g height and width being a factor of 8 (explain why too)

prompt = "A parrot"
height = 512                               
width = 512                                 
inference_steps = 40                       
guidance_scale = 7.5 
seeded_generator = torch.manual_seed(65)

# ['EulerAncestralDiscreteScheduler', 'EulerDiscreteScheduler', 'HeunDiscreteScheduler', 'KDPM2AncestralDiscreteScheduler', 'KDPM2DiscreteScheduler', 'LMSDiscreteScheduler']
scheduler = "LMSDiscreteScheduler"
scheduler_params = {
    "beta_start": 0.00085,
    "beta_end": 0.012,
    "beta_schedule": "scaled_linear",
    "num_train_timesteps": 1000
}

# Generating an image and the visualising its latents
pipeline = StableDiffusionPipeline(prompt, inference_steps, guidance_scale, scheduler=scheduler, scheduler_params={})
image = pipeline.generate(show=True)

latents = pipeline.image_to_latents(image)
pipeline.visualise_noised_latents(latents, pipeline.scheduler, 5)
pipeline.visualise_tokenisation(show_tensor=True, show_attention_mask=True)
pipeline.visualise_scheduler()
pipeline.get_diffusers_schedulers()

  0%|          | 0/40 [00:00<?, ?it/s]c:\Users\diogo\AppData\Local\Programs\Python\Python312\Lib\site-packages\diffusers\models\attention_processor.py:2358: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  hidden_states = F.scaled_dot_product_attention(
  0%|          | 0/40 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [6]:
pipeline.scheduler.timesteps

tensor([999.0000, 988.9091, 978.8182, 968.7273, 958.6364, 948.5455, 938.4545,
        928.3636, 918.2727, 908.1818, 898.0909, 888.0000, 877.9091, 867.8182,
        857.7273, 847.6364, 837.5455, 827.4545, 817.3636, 807.2727, 797.1818,
        787.0909, 777.0000, 766.9091, 756.8182, 746.7273, 736.6364, 726.5455,
        716.4545, 706.3636, 696.2727, 686.1818, 676.0909, 666.0000, 655.9091,
        645.8182, 635.7273, 625.6364, 615.5455, 605.4545, 595.3636, 585.2727,
        575.1818, 565.0909, 555.0000, 544.9091, 534.8182, 524.7273, 514.6364,
        504.5454, 494.4546, 484.3636, 474.2727, 464.1818, 454.0909, 444.0000,
        433.9091, 423.8182, 413.7273, 403.6364, 393.5454, 383.4546, 373.3636,
        363.2727, 353.1818, 343.0909, 333.0000, 322.9091, 312.8182, 302.7273,
        292.6364, 282.5454, 272.4546, 262.3636, 252.2727, 242.1818, 232.0909,
        222.0000, 211.9091, 201.8182, 191.7273, 181.6364, 171.5455, 161.4545,
        151.3636, 141.2727, 131.1818, 121.0909, 111.0000, 100.90

In [ ]:
# TODO add this function as a callback that can be triggered whenever - check if there are any other callbacks that can be used

def show_gpu_usage():
    if torch.cuda.is_available():
        megabyte = 1024 ** 2
        gpu_memory_allocated = torch.cuda.memory_allocated(0) / megabyte
        gpu_memory_reserved = torch.cuda.memory_reserved(0) / megabyte
        print(f"Memory Allocated: {gpu_memory_allocated:.2f} MB")
        print(f"Memory Reserved: {gpu_memory_reserved:.2f} MB")
    else:
        print("CUDA is not currently in use.")

In [ ]:
# TODO split into a visualiser class, that has all the visualisation functions
# TODO selection of the custom networks along with training the

# TODO add to the pipeline but change format and colour - comment code and understand it
    # # Plot some test embeddings
    # test_time_embeddings = get_timestep_embedding(
    #     torch.arange(0,1000), embedding_dim=60
    # )
    # fig, ax = plt.subplots(1,1,figsize=(6,5))
    # sns.heatmap(test_time_embeddings, cmap='bwr', ax=ax)
    # ax.set_xlabel('Embedding Dimension')
    # ax.set_ylabel('Timestep')
    # plt.show()